# Error with `choropleth` function in `folium` module

I encounter an error using `folium` module, making a choropleth map. Here is a reproductible example of the problem.

First, I load GeoJSON file of New-York boroughs.

In [125]:
import json
geo = json.load(open("newyork-boroughs.geojson"))

## Map with markers

When I create a classical map, with markers, they seem to be at the right place.

In [99]:
import folium

loc = [40.71427, -74.00597]
map = folium.Map(location = loc, zoom_start = 11)
for b in geo["features"]:
    folium.Marker(b["properties"]["coordinates"], popup = b["properties"]["BoroName"]).add_to(map)
map

## Choropleth map

I create a `DataFrame` with the borough name and area.

In [126]:
import pandas
import numpy

df = pandas.DataFrame({
    "Borough": [b["properties"]["BoroName"] for b in geo["features"]], 
    "Code": [b["properties"]["BoroCode"] for b in geo["features"]], 
    "Area" : numpy.round([b["properties"]["Shape_Area"] / 1e08 for b in geo["features"]])
})
df

,Area,Borough,Code
0,16.0,Staten Island,5
1,30.0,Queens,4
2,20.0,Brooklyn,3
3,6.0,Manhattan,1
4,12.0,Bronx,2


And I create a choropleth map, with a color according to the area. I choose the borough name to join the data to the geojson.

But, the values are not the good one... Queens is (normally) the biggest borough, and Manhattan the smallest.

In [127]:
cho = folium.Map(location = loc, zoom_start = 10)
cho.choropleth(geo_data = geo, key_on = "feature.properties.BoroName",
               data = df, columns = ["Borough", "Area"],
               fill_color = "Spectral")
cho

In [105]:
geo["features"][0].keys()

dict_keys(['type', 'geometry', 'properties', 'id'])

In [129]:
cho = folium.Map(location = loc, zoom_start = 10)
# Dans le geojson, voila les geometries
# 0 : Bronx
# 1 : Brooklyn
# 2 : Manhattan
# 3 : Queens
# 4 : Staten Island

# Et les properties
# 0 : Staten Island
# 1 : Queens
# 2 : Brooklyn
# 3 : Manhattan
# 4 : Bronx


import copy

geo2 = copy.deepcopy(geo)
geo2["features"][0]["geometry"] = copy.deepcopy(geo["features"][4]["geometry"])
geo2["features"][1]["geometry"] = copy.deepcopy(geo["features"][3]["geometry"])
geo2["features"][2]["geometry"] = copy.deepcopy(geo["features"][1]["geometry"])
geo2["features"][3]["geometry"] = copy.deepcopy(geo["features"][2]["geometry"])
geo2["features"][4]["geometry"] = copy.deepcopy(geo["features"][0]["geometry"])

cho.choropleth(geo_data = geo2, key_on = "feature.properties.BoroName",
               data = df, columns = ["Borough", "Area"],
               fill_color = "Spectral")
cho

In [130]:
fichier = open("../../ny-ok.geojson", "w")
fichier.write(json.dumps(geo2))

123238